In [1]:
# example of pix2pix gan for satellite to map image-to-image translation
import numpy as np
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

In [56]:
import tifffile as tiff
import gc
def untile(im):
    nim=[]
    n=5
    for i in range(n):
        for j in range(n):
            nim.append(im[:,:,(j)*1024:(j+1)*1024,(i)*1024:(i+1)*1024])
    return nim

def selmax(im):
    nim=np.zeros((25,4,1024,1024))
    num=0
    for i in im:
        nim[num]=i[0]
        for j in i:
            if np.mean(j[0])>np.mean(nim[num][0]):
                nim[num]=j
        num+=1
    return np.transpose(nim,(0,2,3,1))

dataim=[tiff.imread("../Documents/Microscopie/2022-04-21_BioSamples_Acquisition/01_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread("../Documents/Microscopie/2022-04-21_BioSamples_Acquisition/05_NDD+interne_5-5_1024-1024_5s.lsm"),
        tiff.imread("../Documents/Microscopie/2022-04-21_BioSamples_Acquisition/07_NDD+interne_5-5_1024-1024_5s.lsm")]
imgs=None
for im in dataim:
    im2=untile(im)
    im3=selmax(im2).astype("float32")
    if imgs is None:
        imgs=im3
    else:imgs=np.concatenate((imgs,im3),axis=0)
    del im2,im3
del dataim
gc.collect()

86811

In [57]:
imshape=np.shape(np.expand_dims(imgs[0,:,:,0],axis=-1))
nddr=np.expand_dims(imgs[...,0],axis=3)
nddg=np.expand_dims(imgs[...,1],axis=3)
intg=np.expand_dims(imgs[...,2],axis=3)
intr=np.expand_dims(imgs[...,3],axis=3)
del imgs
gc.collect()
X=np.concatenate((intg,intr),axis=0)
Y=np.concatenate((nddg,nddr),axis=0)
del nddr,nddg,intg,intr
gc.collect()
x_test,y_test=X,Y
del X,Y
gc.collect()

0

In [58]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class Tile:
    '''
    Usage    : Tile(image,tilesize,overlap,verbose)
    image    : Array of shape [height,width,canals]
    tilesize : The output will be of shape [tilesize,tilesize,canals]
    overlap  : Amount of pixel common between two consecutive tiles
    ================================================================
    Methods  :
    padding(mode="reflect"): Returns a padded image via numpy.pad usage
    tilegeneration()       : Returns a list parts of original image(tiles)
    tilevis(concat=True)   : Plots the tiles list for the image
    detmask()              : Returns the mask of overlapping areas
    reconstruct()          : Builds the image back out of the tile list   
    '''
    
    def __init__(self, image, tilesize=None,overlap=None,verbose=False):
        
        self.im = image
        self.s  = np.shape(self.im)[:2]
        if tilesize is None:tilesize=self.s[0]//4
        self.t  = tilesize
        if overlap is None :overlap =tilesize//4
        self.o  = overlap
        self.verbose=verbose
        self.verdata()
        self.to = self.t-self.o
        self.nbt= [(self.s[0]-self.o)//self.to+1,(self.s[1]-self.o)//self.to+1]
        self.p  = None
        self.pIm= None
        self.tl = None
        
    def verdata(self):
        if (np.shape(self.im)[0]%2)!=0:
            if self.verbose:print("Changing image to an even dimension value, one pixel was removed on axis zero.")
            self.im=self.im[:-1]
        if (np.shape(self.im)[1]%2)!=0:
            if self.verbose:print("Changing image to an even dimension value, one pixel was removed on axis one.")
            self.im=self.im[:,:-1]
        if self.o%2!=0:
            if self.verbose:print("Please use even value for overlap, overlap was reduced by one.")
            self.o-=1
        if self.t>np.shape(self.im)[0] or self.t>np.shape(self.im)[1]:
            if self.verbose:print("You probably didn't do what you want, you will just have one padded image. (Tile size larger than image)")
                      
    def detpad(self):
        psize=[(self.nbt[0]*self.to+self.o)-self.s[0],(self.nbt[1]*self.to+self.o)-self.s[1]]
        for i in range(len(psize)):
            if  psize[i]==self.to:
                psize[i]%=self.to
                self.nbt[i]-=1
        return psize
    
    def padding(self,mode="reflect"):
        if self.p is None:
            self.p=self.detpad()
        padval=int(self.p[0]/2),int(self.p[1]/2)
        pIm=np.pad(self.im,((padval[0],padval[0]),(padval[1],padval[1]),(0,0)),mode=mode)
        del padval
        gc.collect()
        return pIm
    
    def tilegeneration(self):
        if self.pIm is None:
            self.pIm=self.padding()
        tiles=[]
        for i in range (self.nbt[0]):
            for j in range (self.nbt[1]):
                tiles.append(self.pIm[i*self.to:i*self.to+self.t,j*self.to:j*self.to+self.t])
        return tiles
    
    def tilevis(self,concat=True):
        if self.tl is None:
            self.tl=self.tilegeneration()
        plt.figure(figsize =( 3*self.nbt[1], 3*self.nbt[0]))
        if not concat:
            for i in range(self.nbt[0]):
                for j in range(self.nbt[1]):
                    ax = plt.subplot(self.nbt[0], self.nbt[1], i*self.nbt[1]+j+1)
                    ax.axis("off")
                    plt.imshow(self.tl[i*self.nbt[1]+j])
        else:
            im=[]
            for i in range(self.nbt[0]):
                line=[]
                for j in range(self.nbt[1]):
                    if j==0: line=self.tl[i*self.nbt[1]+j]
                    else: line = np.concatenate((line,self.tl[i*self.nbt[1]+j]),axis=1)
                if i==0:im=line
                else:im=np.concatenate((im,line),axis=0)
            plt.imshow(im[...,0])
        del im
        gc.collect()
        plt.show()
        
    
    def detmask(self):
        if self.pIm is None:
            self.pIm=self.padding()
        mask=np.zeros(np.shape(self.pIm)[:2])
        for i in range (self.nbt[0]):
            for j in range (self.nbt[1]):
                mask[i*self.to:i*self.to+self.t,j*self.to:j*self.to+self.t]+=np.ones((self.t,self.t))
        return mask[int(self.p[0]/2):self.s[0]+int(self.p[0]/2),int(self.p[1]/2):self.s[1]+int(self.p[1]/2)]
    
    def reconstruct(self,tl=None):
        if tl is None:
            tl=self.tilegeneration()
        reim=np.zeros(np.shape(self.pIm))
        for i in range (self.nbt[0]):
            for j in range (self.nbt[1]):
                reim[i*self.to:i*self.to+self.t,j*self.to:j*self.to+self.t]+=tl[i*self.nbt[1]+j]
        reim=reim[int(self.p[0]/2):self.s[0]+int(self.p[0]/2),int(self.p[1]/2):self.s[1]+int(self.p[1]/2)]
        mask=self.detmask()
        for i in range(np.shape(reim)[2]):
            reim[...,i]/=mask
        if np.max(reim)<=1:
          reim=(reim*255)
        return reim.astype(int)


In [59]:
def tiling(dataset,tilesize=256,overlap=64):
    
    tiled=[]
    for i in dataset:
        tempi=Tile(i,tilesize,overlap)
        tiled.append(tempi.tilegeneration())
    del tempi
    gc.collect()
    return np.reshape(tiled,(-1,tilesize,tilesize,np.shape(dataset)[-1]))
tilesize=256
overlap=64
x_test,y_test=tiling(x_test),tiling(y_test)

In [60]:
print(np.min(x_test),np.max(x_test))
x_test_processed=(((x_test/255.0*2)-1).copy()).astype("float32")
y_test_processed=(((y_test/255.0*2)-1).copy()).astype("float32")
print(np.min(x_test),np.max(x_test))
print(np.min(x_test_processed),np.max(x_test_processed))

0.0 255.0
0.0 255.0
-1.0 1.0


In [68]:
import tensorflow as tf
g_model = tf.keras.models.load_model('champipix_conv3_0')


In [41]:
x_res=g_model.predict(x_test_processed[:10],verbose=True)

1/1 [==============================] - 1s 555ms/step


In [154]:
tinfo=Tile(np.zeros((1024,1024,1)),tilesize,overlap)
xtrec=[]
tinfo.pIm=tinfo.padding()
nim=tinfo.nbt[0]*tinfo.nbt[1]
print(np.shape(x_res))
for i in range(len(x_res)//nim):
    xtrec.append(tinfo.reconstruct(x_res[nim*i:nim*(i+1)]))
print(np.shape(xtrec))
print(np.max(xtrec))

(1250, 256, 256, 1)
(50, 1024, 1024, 1)
235


In [157]:
fres=np.zeros([2,5120,5120,1])
for k in range(2):
    for j in range (5):
        for i in range (5):
            fres[k,i*1024:i*1024+1024,j*1024:j*1024+1024]=(xtrec[25*k+5*j+i])

In [158]:
if True:
    from PIL import Image
    for i in range(len(fres)):
        im=np.concatenate((fres[i],fres[i],fres[i]),axis=2)
        im=Image.fromarray((255*(im-np.min(im))/(np.max(im)-np.min(im))).astype("uint8"))
        im.save("champipredoct%d.png"%i)
        del im
gc.collect()

24

In [66]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [ ]:

n_samples=30
x_valid_processed=np.copy(x_test_processed).astype("float32")
y_valid_processed=np.copy(y_test_processed).astype("float32")
validset = [x_valid_processed,y_valid_processed]
g_model = tf.keras.models.load_model('champipix_conv3_gen_3') 
[X_realA, X_realB], _ = generate_real_samples(validset, n_samples, 1)
# generate a batch of fake samples
X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
# scale all pixels from [-1,1] to [0,1]
print(np.min(X_realA),np.max(X_realA))
print(np.min(X_realB),np.max(X_realB))
print(np.min(X_fakeB),np.max(X_fakeB))
X_realA = (X_realA + 1) / 2.0
X_realB = (X_realB + 1) / 2.0
X_fakeB = (X_fakeB + 1) / 2.0
# plot generated target image
for i in range(10):
    pyplot.figure(figsize=(20,20))
    pyplot.axis('off')
    pyplot.imshow(X_realA[i].astype("float32").squeeze(),cmap="gray")  
    pyplot.figure(figsize=(20,20))
    pyplot.axis('off')
    pyplot.imshow(X_fakeB[i].astype("float32").squeeze(),cmap="gray")  
    pyplot.figure(figsize=(20,20))
    pyplot.axis('off')
    pyplot.imshow(X_realB[i].astype("float32").squeeze(),cmap="gray")
# plot real target image

In [ ]:
n_samples=20
x_valid_processed=np.copy(((x_test/255.0)*2)-1).astype("float32")
y_valid_processed=np.copy(((y_test/255.0)*2)-1).astype("float32")
validset = [x_valid_processed[:20],y_valid_processed[:20]]

#g_model = tf.keras.models.load_model('gdrive/MyDrive/champipix_conv3') 
[X_realA, X_realB], _ = generate_real_samples(validset, n_samples, 1)
# generate a batch of fake samples
X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
# scale all pixels from [-1,1] to [0,1]
print(np.min(X_realA),np.max(X_realA))
print(np.min(X_realB),np.max(X_realB))
print(np.min(X_fakeB),np.max(X_fakeB))
X_realA = (X_realA + 1) / 2.0
X_realB = (X_realB + 1) / 2.0
X_fakeB = (X_fakeB + 1) / 2.0
# plot generated target image
for i in range(10,20):
  pyplot.figure(figsize=(20,20))
  pyplot.axis('off')
  pyplot.imshow(X_realA[i].astype("float32").squeeze(),cmap="gray")  
  pyplot.figure(figsize=(20,20))
  pyplot.axis('off')
  pyplot.imshow(X_fakeB[i].astype("float32").squeeze(),cmap="gray")  
  pyplot.figure(figsize=(20,20))
  pyplot.axis('off')
  pyplot.imshow(X_realB[i].astype("float32").squeeze(),cmap="gray")
# plot real target image


In [ ]:
for i in range(10):
    plt.figure(figsize=(20,20))
    plt.imshow((x_res[i]+1)/2.0,cmap="gray")
    plt.figure(figsize=(20,20))
    plt.imshow((x_test_processed[i]+1)/2.0,cmap="gray")
    